In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('C:\\Users\\Biplob\\OneDrive\\Desktop\\twitter\\TwitterSentiment\\input\\training.1600000.processed.noemoticon.csv',
                 encoding = 'latin',
                 names=["sentiment", "id", "date", "query", "twitter_handler", "tweet"])
df.head()

,sentiment,id,date,query,twitter_handler,tweet
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [3]:
df.drop(["id", "date", "query", "twitter_handler"], axis=1, inplace=True)

In [4]:
df.head()

,sentiment,tweet
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,is upset that he can't update his Facebook by ...
2,0,@Kenichan I dived many times for the ball. Man...
3,0,my whole body feels itchy and like its on fire
4,0,"@nationwideclass no, it's not behaving at all...."


In [5]:
df["sentiment"].unique()

array([0, 4], dtype=int64)

In [6]:
sentiment = {
    0: "Negative",
    4: "Positive"
}

def label_converter(label):
    return sentiment[label]

df["sentiment"] = df.sentiment.apply(lambda x : label_converter(x))

In [7]:
df.head()

,sentiment,tweet
0,Negative,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,Negative,is upset that he can't update his Facebook by ...
2,Negative,@Kenichan I dived many times for the ball. Man...
3,Negative,my whole body feels itchy and like its on fire
4,Negative,"@nationwideclass no, it's not behaving at all...."


In [8]:
from nltk.corpus import stopwords
stop_words = set(stopwords.words("english"))

def remove_stopwords(data):
    filtered_data = [word.lower() for word in data.split() if word.lower() not in stop_words]
    return " ".join(filtered_data)
df["tweet"] = df["tweet"].map(remove_stopwords)

In [9]:
df.head()

,sentiment,tweet
0,Negative,"@switchfoot http://twitpic.com/2y1zl - awww, t..."
1,Negative,upset can't update facebook texting it... migh...
2,Negative,@kenichan dived many times ball. managed save ...
3,Negative,whole body feels itchy like fire
4,Negative,"@nationwideclass no, behaving all. i'm mad. he..."


In [10]:
def clean_data(name):
    # Replace email addresses with 'email'
    processed = name.str.replace(r'^.+@', '')
    processed = processed.str.replace(r'[^\w\d\s]', ' ')
    # Replace whitespace between terms with a single space
    processed = processed.str.replace(r'\s+', ' ')
    # Remove leading and trailing whitespace
    processed = processed.str.replace(r'^\s+|\s+?$', '')
    # change words to lower case - Hello, HELLO, hello are all the same word
    processed = processed.str.lower()
    return processed

In [11]:
df["tweet"] = clean_data(df["tweet"])

In [12]:
df.head()

,sentiment,tweet
0,Negative,switchfoot http twitpic com 2y1zl awww that s ...
1,Negative,upset can t update facebook texting it might c...
2,Negative,kenichan dived many times ball managed save 50...
3,Negative,whole body feels itchy like fire
4,Negative,nationwideclass no behaving all i m mad here c...


In [13]:
from sklearn import model_selection

if __name__ == "__main__":
    df["kfold"] = -1
    df["sentiment"] = df.sentiment.apply(lambda x: 0 if x == "Negative" else 1)
    df = df.sample(frac=1).reset_index(drop=True)
    y = df.sentiment.values
    kf = model_selection.StratifiedKFold(n_splits=5)
    for f, (t_, v_) in enumerate(kf.split(X=df, y=y)):
        df.loc[v_, "kfold"] = f
    df.to_csv("C:\\Users\\Biplob\\OneDrive\\Desktop\\twitter\\TwitterSentiment\\input\\twitter_fold.csv", index=False)